# Comparación de Clustering DBSCAN y HDBSCAN

Después de completar este laboratorio, podrás:

Usar scikit-learn para implementar modelos de clustering DBSCAN y HDBSCAN en datos reales.

Comparar el desempeño de ambos modelos.

En este laboratorio, crearás dos modelos de clustering utilizando datos recopilados por StatCan, que contienen los nombres, tipos y ubicaciones de instalaciones culturales y artísticas en Canadá.
Nos centraremos en las ubicaciones de los museos proporcionadas en todo Canadá.

Fuente de los datos: Base de Datos Abierta de Instalaciones Culturales y Artísticas (ODCAF)

Una colección de datos abiertos que contiene los nombres, tipos y ubicaciones de instalaciones culturales y artísticas en Canadá.
Se publica bajo la Licencia de Gobierno Abierto - Canadá.
Los diferentes tipos de instalaciones se etiquetan bajo 'ODCAF_Facility_Type'.

Página principal:

https://www.statcan.gc.ca/en/lode/databases/odcaf

Enlace al archivo zip:

https://www150.statcan.gc.ca/n1/en/pub/21-26-0001/2020001/ODCAF_V1.0.zip?st=brOCT3Ry

In [27]:
!pip install numpy==2.2.0
!pip install pandas==2.2.3
!pip install scikit-learn==1.6.0
!pip install matplotlib==3.9.3
!pip install hdbscan==0.8.40
!pip install geopandas==1.0.1
!pip install contextily==1.6.2
!pip install shapely==2.0.6

  Using cached hdbscan-0.8.40.tar.gz (6.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build hdbscan


  error: subprocess-exited-with-error
  
  × Building wheel for hdbscan (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      C:\Users\Josue Santana Robled\AppData\Local\Temp\pip-build-env-l3j8ra48\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      C:\Users\Josue Santana Robled\AppData\Local\Temp\pip-build-env-l3j8ra48\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\Josue Santana Robled\AppData\Local\Temp\pip-build-env-l3j8ra48\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn.preprocessing import StandardScaler

# geographical tools
import geopandas as gpd  # pandas dataframe-like geodataframes for geographical data
import contextily as ctx  # used for obtianing a basemap of Canada
from shapely.geometry import Point

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'hdbscan'

Descargar el mapa de Canadá como referencia

Para obtener un contexto adecuado del resultado final de este laboratorio, necesitas un mapa de referencia de Canadá.
Ejecuta la celda a continuación para extraerlo en este entorno del laboratorio.

In [3]:
import requests
import zipfile
import io
import os

# URL of the ZIP file on the cloud server
zip_file_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/YcUk-ytgrPkmvZAh5bf7zA/Canada.zip'

# Directory to save the extracted TIFF file
output_dir = './'
os.makedirs(output_dir, exist_ok=True)

# Step 1: Download the ZIP file
response = requests.get(zip_file_url)
response.raise_for_status()  # Ensure the request was successful
# Step 2: Open the ZIP file in memory
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    # Step 3: Iterate over the files in the ZIP
    for file_name in zip_ref.namelist():
        if file_name.endswith('.tif'):  # Check if it's a TIFF file
            # Step 4: Extract the TIFF file
            zip_ref.extract(file_name, output_dir)
            print(f"Downloaded and extracted: {file_name}")

Downloaded and extracted: Canada.tif


## Incluir una función de visualización

Se proporciona el código de una función auxiliar para ayudarte a graficar tus resultados.
Aunque no necesitas preocuparte por los detalles, es bastante instructivo, ya que utiliza un dataframe de geopandas y un basemap para dibujar los puntos de los clústeres coloreados sobre un mapa de Canadá.

In [6]:
# Write a function that plots clustered locations and overlays them on a basemap.

def plot_clustered_locations(df,  title='Museums Clustered by Proximity'):
    """
    Plots clustered locations and overlays on a basemap.

    Parameters:
    - df: DataFrame containing 'Latitude', 'Longitude', and 'Cluster' columns
    - title: str, title of the plot
    """

    # Load the coordinates intto a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), crs="EPSG:4326")

    # Reproject to Web Mercator to align with basemap
    gdf = gdf.to_crs(epsg=3857)

    # Create the plot
    fig, ax = plt.subplots(figsize=(15, 10))

    # Separate non-noise, or clustered points from noise, or unclustered points
    non_noise = gdf[gdf['Cluster'] != -1]
    noise = gdf[gdf['Cluster'] == -1]

    # Plot noise points
    noise.plot(ax=ax, color='k', markersize=30, ec='r', alpha=1, label='Noise')

    # Plot clustered points, colured by 'Cluster' number
    non_noise.plot(ax=ax, column='Cluster', cmap='tab10', markersize=30, ec='k', legend=False, alpha=0.6)

    # Add basemap of  Canada
    ctx.add_basemap(ax, source='./Canada.tif', zoom=4)

    # Format plot
    plt.title(title, )
    plt.xlabel('Longitude', )
    plt.ylabel('Latitude', )
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()

    # Show the plot
    plt.show()

## Explora los datos y extrae lo que necesites de ellos

In [8]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/r-maSj5Yegvw2sJraT15FA/ODCAF-v1-0.csv'
df = pd.read_csv(url, encoding = "ISO-8859-1")
df.head()

,Index,Facility_Name,Source_Facility_Type,ODCAF_Facility_Type,Provider,Unit,Street_No,Street_Name,Postal_Code,City,Prov_Terr,Source_Format_Address,CSD_Name,CSDUID,PRUID,Latitude,Longitude
0,1,#Hashtag Gallery,..,gallery,toronto,..,801,dundas st w,M6J 1V2,toronto,on,801 dundas st w,Toronto,3520005,35,43.65169472,-79.40803272
1,2,'Ksan Historical Village & Museum,historic site-building or park,museum,canadian museums association,..,1500,62 hwy,V0J 1Y0,hazelton,bc,1500 hwy 62 hazelton british columbia v0j 1y0 ...,Hazelton,5949022,59,55.2645508,-127.6428124
2,3,'School Days' Museum,community/regional museum,museum,canadian museums association,..,427,queen st,E3B 5R6,fredericton,nb,427 queen st fredericton new brunswick e3b 5r6...,Fredericton,1310032,13,45.963283,-66.6419017
3,4,10 Austin Street,built heritage properties,heritage or historic site,moncton,..,10,austin st,E1C 1Z6,moncton,nb,10 austin st,Moncton,1307022,13,46.09247776,-64.78022946
4,5,10 Gates Dancing Inc.,arts,miscellaneous,ottawa,..,..,..,..,ottawa,on,..,Ottawa,3506008,35,45.40856224,-75.71536766


Ejercicio 1

Explora la tabla. ¿Cómo se representan los valores faltantes (missing values) en este conjunto de datos?

Las cadenas que consisten en dos puntos '..' indican valores faltantes.
También podrían existir campos vacíos o NaN.

### Ejercicio 2

Muestra los tipos de instalaciones y sus conteos.

In [12]:
df.ODCAF_Facility_Type.value_counts()

ODCAF_Facility_Type
library or archives                     3013
museum                                  1938
gallery                                  810
heritage or historic site                620
theatre/performance and concert hall     583
festival site                            346
miscellaneous                            343
art or cultural centre                   225
artist                                    94
Name: count, dtype: int64

### Ejercicio 3
Filtra los datos para incluir únicamente museos.
Verifica tus resultados. ¿Obtuviste tantos como esperabas?

In [13]:
df = df[df.ODCAF_Facility_Type == 'museum']
df.ODCAF_Facility_Type.value_counts()

ODCAF_Facility_Type
museum    1938
Name: count, dtype: int64

### Ejercicio 4
Selecciona únicamente las características de Latitud y Longitud como entradas para nuestro problema de clustering.
Además, muestra información sobre las coordenadas, como los conteos y los tipos de datos.

In [14]:
df = df[['Latitude', 'Longitude']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1938 entries, 1 to 7969
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Latitude   1938 non-null   object
 1   Longitude  1938 non-null   object
dtypes: object(2)
memory usage: 45.4+ KB


### Ejercicio 5
Necesitaremos que estas coordenadas sean floats, no objetos.
Elimina cualquier museo que no tenga coordenadas y convierte las coordenadas restantes a floats.

In [15]:
df = df[df.Latitude!='..']

df[['Latitude','Longitude']] = df[['Latitude','Longitude']].astype('float')

## Construir un modelo DBSCAN
Escala correctamente las coordenadas para DBSCAN (ya que DBSCAN es sensible a la escala)

In [16]:
# In this case we know how to scale the coordinates. Using standardization would be an error becaues we aren't using the full range of the lat/lng coordinates.
# Since latitude has a range of +/- 90 degrees and longitude ranges from 0 to 360 degrees, the correct scaling is to double the longitude coordinates (or half the Latitudes)
coords_scaled = df.copy()
coords_scaled["Latitude"] = 2*coords_scaled["Latitude"]

### Aplicar DBSCAN con distancia Euclidiana a las coordenadas escaladas

En este caso, los parámetros de vecindad razonables ya están seleccionados para ti.
Siéntete libre de experimentar.

In [17]:
min_samples=3 # minimum number of samples needed to form a neighbourhood
eps=1.0 # neighbourhood search radius
metric='euclidean' # distance measure

dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric=metric).fit(coords_scaled)

### Agregar etiquetas de clúster al DataFrame

In [18]:
df['Cluster'] = dbscan.fit_predict(coords_scaled)  # Assign the cluster labels

# Display the size of each cluster
df['Cluster'].value_counts()

Cluster
 4     701
 2     192
 1     181
 7     134
 3      94
-1      79
 6      30
 10     27
 8      21
 11     15
 15     13
 20     11
 16     10
 19      9
 27      8
 12      7
 26      6
 5       6
 24      6
 28      6
 14      6
 18      6
 13      4
 9       4
 22      4
 0       3
 23      3
 21      3
 17      3
 25      3
 29      3
 31      3
 30      3
 32      3
Name: count, dtype: int64

Como puedes ver, hay dos clústeres relativamente grandes y 79 puntos etiquetados como ruido (-1).

### Grafica los museos en un basemap de Canadá, coloreados según la etiqueta del clúster.

In [ ]:
plot_clustered_locations(df, title='Museums Clustered by Proximity')

- ¿Qué ves?
- ¿Cuál es el tamaño del grupo más pequeño?
- ¿Crees que los grupos tienen sentido en términos de lo que esperas ver?
- ¿Crees que debería haber más grupos en algunas regiones? ¿Por qué?

Una cosa clave para notar aquí es que los grupos no tienen una densidad uniforme.

Por ejemplo, los puntos están bastante densamente agrupados en algunas regiones, pero son relativamente dispersos en otras.

DBSCAN aglomera los grupos vecinos cuando están lo suficientemente cerca.

Veamos cómo funciona un algoritmo de agrupamiento basado en densidad jerárquica como HDBSCAN.

## Construir un modelo de clustering HDBSCAN
En esta etapa, ya has cargado tus datos y extraído las coordenadas de los museos en un dataframe, df.

También has almacenado las coordenadas debidamente escaladas en el array 'coords_scaled'.

Solo queda:

Ajustar y transformar HDBSCAN a tus coordenadas escaladas
Extraer las etiquetas de los clusters
Graficar los resultados en el mismo mapa base que antes
Se han seleccionado parámetros razonables de HDBSCAN para que comiences.

#### Inicializa un módelo HDBSCAN

In [19]:
min_samples=None
min_cluster_size=3
hdb = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric='euclidean')  # You can adjust parameters as needed

NameError: name 'hdbscan' is not defined

## Ejercicio 6. Asigna las etiquetas de los clusters a tu dataframe con coordenadas sin escalar y muestra el conteo de cada etiqueta de cluster.

In [ ]:
df['Cluster'] = hdb.fit_predict(coords_scaled)  # Another way to assign the labels

# Display the size of each cluster
df['Cluster'].value_counts()

Como puedes ver, a diferencia del caso de DBSCAN, los clusters tienen un tamaño bastante uniforme, aunque se identifica una cantidad considerable de ruido.

## Ejercicio 7. Grafica los museos agrupados jerárquicamente en un mapa base de Canadá, coloreados según la etiqueta de cluster.


In [25]:
plot_clustered_locations(df, title='Museums Hierarchically Clustered by Proximity')

NameError: name 'gpd' is not defined

### Comentarios finales

Observa detenidamente el mapa.

- ¿Qué es diferente en estos resultados comparados con DBSCAN?
Podría parecer que hay más puntos identificados como ruido, ¿pero es realmente así?
- ¿Puedes ver las variaciones en la densidad que HDBSCAN captura?
En la práctica, querrías investigar mucho más a fondo, pero al menos aquí tienes la idea.